# Report for 2D Project Physical World and Digital World

Cohort: # 9

Team No.: # 6

Members:
* Tan Jun Yong (1003057)
* Lee Hang Wee (1002727)
* Lee Jia Yi (1003137)
* Qing Renjie (1002969)
* Wei Letong (1002965)

# Introduction

The conventional thermometer takes approximately more than 100s to be able to reach quasi-thermal equilibrium with the water bath. In order to reduce the time taken, a linear regression model that takes into account the heat capacity and thermal conductance is used.

Governing Equation, Tsensor = A(exp(-time/tau)) + Twater, where tau is the time constant and is determined through dividing the heat capacity by thermal conductance, and A is (Troom - Twater); Troom is the room temperature, assuming it is the initial temperature the temperature senses.

Therefore, the purpose of this experiment is to test out the hypothesis and regression model on whether it is possible to accurately measure the temperature of an object faster than a conventional thermometer. 



# Description of Data from Experiment
In order to come up with a regression model, an experiment with a temperature of 44.3 degrees celsius has been set up to collect the datasets using the Raspberry Pi to ascertain the relationship with the governing equation.



## Data Collection

Using the thermometer sensor that was connected to the Raspberry Pi, we read the values detected by the temperature sensor within a time a period 120 seconds and over an average time interval of 1.5 seconds. 

These values are also written into a text file, with each line indicating the time elapsed (from its first reading of the room temperature) and the temperature corresponding to the time values.

The text file is then imported to an Excel file where the first column represents the time values and the second column representing the corresponding temperature values.

In [ ]:
#Code to activate temperature sensor 

import os
import glob
import time
import numpy as np
from sklearn import linear_model
import math

os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')

base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'

def read_temp_raw():
    f = open(device_file, 'r')
    lines = f.readlines()
    f.close()
    return lines

def read_temp():
    lines = read_temp_raw()
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw()
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        return temp_c
    
    
# code to write temperature values into a list of nested lists
start_time=time.time()
elapsed_time=time.time() - start_time

cries=[]

while elapsed_time <= 25:
    reading_i= read_temp()
    subcries=[elapsed_time,reading_i]
    cries.append(subcries)
    time.sleep(0.5)
    elapsed_time = time.time() - start_time
    

## Data Preparation

As mentioned above, the exponential equation that shows how temperature of the sensor relates to time is: Tsensor = A(exp(-time/tau)) + Twater, where tau: time constant and is determined through dividing the heat capacity by thermal conductance, and A: (Troom - Twater).

To predict the temperature of the water using this equation, we differentiate Tsensor with respect to time and we obtain the governing equation that will apply to any temperature level of the water: 

Tsensor/dtime = (-A/tau)(exp(-time/tau)) = (-(Troom - Twater)/tau)(exp(-time/tau))

To perform linear regression to our values, we modify the differentiated equation above to give us:

ln(dTsensor/dtime) = (-1/tau)t + ln((Twater-Troom)/tau), where the we perfomr linear regression of ln(dTsensor/dtime) against t

The equation ln(dTsensor/dtime) = (-1/tau)t + ln((Twater-Troom)/tau) allows us to find Twater by assessing the y-intercept (ln((Twater-Troom)/tau)) and gradient (-1/tau) of the linear model plotted:

Twater=(exp(intercept)/(-(-1/tau)))+Troom

Hence, we have to calculate the values of ln(dTsensor/dtime) ≈ ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) 


## Data Format

After exporting the data from the excel file as an array, we calculated the values of ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) and added them to the end of each row of the array. 
Then, we delted the values of Ts to plot a graph of ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) against time.

To increase the accuracy of our model, we only plotted linear regression for the first 15% of the values in the array, as only these values provided a linear model which we are able to find the y-intercept and gradient to calculate and Twater (before the temperature sensor detects it).
 

# Linear Regression to find model.coef and model.intercept

After calculating the values for ln((Tsensor2 - Tsensor1)/(dt1 - dt2)) and deleting the Ts values from the array, our new array (labelled as 'data') now contains an array of the first 15 percent of the time values on the first column and the corresponging values of ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) on the second column.

Using linear regression, we plot a best fit line with the value in the 'data' array and we calculated the value of Twater by manipulating the y-intercept (model.intercept) and gradient (model.coef) of our best fit line.

y-intercept: ln((Twater-Troom)/tau) 
Gradient -1/tau

Therefore, Twater=(exp(intercept)/(-(-1/tau)))+Troom

# Testing Using Instructor's Data

In [76]:
import pandas as pd
import numpy as np
num_test = 3
filename = 'temp_' 
filekey = ['513','96','514'] #has to be string
dataframe = {} 

dataframe = {} # this is to store the data for different temperature, the keys are in filekey
for idx in range(num_test):
    dataframe[filekey[idx]] = pd.read_excel(filename+filekey[idx]+'.xlsx')


def preprocess(df):
    cries=df.values
    cries=cries.tolist()
    i=0
    while i < len(cries):
        if float(cries[i][0])==0:
            cries[i].append(None)
        else:
            yvalue=np.log((float(cries[i][1])-float(cries[i-1][1]))/(float(cries[i][0])-float(cries[i-1][0])))
            cries[i].append(yvalue)
        i=i+1

    old_data=np.array(cries) #after adding the values of ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) 

    big_data=old_data[1:,[0,2]] #deleting values of Ts to plot a graph of ln((Tsensor2 - Tsensor1)/(dt2 - dt1)) against time

#with our values for different water temperature, 
#we observed that plotting linear regression of the first 15 percent of the values improves the accuracy to calculate Twater

    test_size=int(round(len(big_data)*0.15,0)) 
    data=big_data[:test_size,:]
    roomt=old_data[0:1,1:2][0][0] #room temperature value
    together=[data,roomt]
    return together

data_test = {}
for key in filekey:
    data_test[key]=preprocess(dataframe[key])
    






C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in log


In [78]:
import numpy as np
from sklearn import model_selection
from sklearn import linear_model
from sklearn import datasets
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import math

def linear_regression (data):
    
    x_data = data[0][:,0]
    y_data = data[0][:,1]

    x_train=x_data
    y_train=y_data
    
    model=linear_model.LinearRegression()
    x_train=x_train.reshape(-2,1)
    y_train=y_train.reshape (-1,1)

    model.fit(x_train,y_train)   
    
    intercept=model.intercept_[0] 
    coef=model.coef_[0][0]
    
    predictedtemperature=(math.e**(intercept)/(-coef))+data[1]
    return predictedtemperature


predicted = {}
for key in filekey:
    predicted[key]=linear_regression(data_test[key])
    
print (predicted)


{'513': 49.415123252573437, '96': 45.804017490143252, '514': 49.415123252573437}


In [ ]:
# checking accuracy

# first instructor will load the actual temp from a pickle object
import pickle
error_d = {}
accuracy_percent_d = {}

for test in range(num_test):
    filename = 'data_'+filekey[test]+'.pickle'
    with open(filename,'rb') as f:
        final_temp, worst_temp = pickle.load(f)

    # then calculate the error
    error_final = abs(final_temp-predicted[filekey[test]])
    accuracy_final_percent = 100-error_final/final_temp*100
    error_worst = abs(worst_temp-predicted[filekey[test]])
    accuracy_worst_percent = 100-error_worst/worst_temp*100
    
    error_d[filekey[test]] = (error_final, error_worst)
    accuracy_percent_d[filekey[test]] = (accuracy_final_percent, accuracy_worst_percent)

    # displaying the error
    print('===================================')
    print('Testing: {}'.format(filekey[test]))
    print('Predicted Temp: {:.2f}'.format(predicted[filekey[test]]))
    print('Final Sensor Temp: {:.2f}, Alcohol Temp:{:.2f}'.format(final_temp, worst_temp))
    print('Error w.r.t Final Sensor Temp: {:.2f} deg, {:.2f}% accuracy'.format(error_final, accuracy_final_percent))
    print('Error w.r.t Alcohol Temp: {:.2f} deg, {:.2f}% accuracy'.format(error_worst, accuracy_worst_percent))
    
avg_final = sum([ final for final, worst in accuracy_percent_d.values()])/len(error_d.values())
avg_worst = sum([ worst for final, worst in accuracy_percent_d.values()])/len(error_d.values())
print('==============================')
print('Average accuracy for final sensor temp: {:.2f}'.format(avg_final))
print('AVerage accuracy for alcohol temp: {:.2f}'.format(avg_worst))
